In [7]:
!pip install opencv-python
!pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O yolov3.cfg
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true -O coco.names


--2024-11-03 01:34:10--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: 'yolov3.weights'

     0K .......... .......... .......... .......... ..........  0%  177K 22m49s
    50K .......... .......... .......... .......... ..........  0%  187K 22m10s
   100K .......... .......... .......... .......... ..........  0% 3.56M 15m9s
   150K .......... .......... .......... .......... ..........  0% 2.96M 11m41s
   200K .......... .......... .......... .......... ..........  0%  197K 13m27s
   250K .......... .......... .......... .......... ..........  0% 3.54M 11m24s
   300K .......... .......... .......... .......... ..........  0%  198K 12m41s
   350K .......... .......... .......... .......... ..........  0% 3.52M 11m14s
   400K .......... .......... ..

In [ ]:
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow # uncomment this line to run in google colab

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load an image
image = cv2.imread('image.jpg')  # Replace with your image path
height, width, _ = image.shape

# Prepare the image for detection
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Initialize lists to hold detection results
boxes = []
confidences = []
class_ids = []

# Process each output layer
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        # Filter for persons only (class_id 0 corresponds to 'person')
        if confidence > 0.5 and class_id == 0:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Maximum Suppression to suppress overlapping boxes
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw bounding boxes on the image
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label + " " + str(round(confidences[i], 2)), (x, y + 30),
                    cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

# Show the output image with detections
# Process the image (for example, convert to grayscale)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Display the grayscale image using cv2_imshow
# cv2_imshow(gray_image) # uncomment this line to run in google colab
cv2.imshow('gray_image', gray_image) # comment this line to run in google colab

# If you want to display the original image as well:
# cv2_imshow(image) # uncomment this line to run in google colab
cv2.imshow('image', image) # comment this line to run in google colab

cv2.waitKey(0)
cv2.destroyAllWindows()
